# Import New URLs And Store Them In The Bronze Table

In [0]:
import requests
import zipfile
import io
import pandas as pd
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, monotonically_increasing_id, lit

# Step 1: Get the URL of the latest GKG CSV zip file from the provided link
response = requests.get("http://data.gdeltproject.org/gdeltv2/lastupdate.txt")
latest_gkg_zip_url = None
for line in response.text.split("\n"):
    if line.endswith(".gkg.csv.zip"):
        latest_gkg_zip_url = line.split()[-1]
        break

# Step 2: Download the zip file
zip_response = requests.get(latest_gkg_zip_url)
zip_file = zipfile.ZipFile(io.BytesIO(zip_response.content))

# Step 3: Extract the CSV file from the zip
csv_file_name = zip_file.namelist()[0]  # Since there is only one CSV file in the zip
csv_file = zip_file.read(csv_file_name)

# Step 4: Convert the CSV content to a DataFrame
df = pd.read_csv(io.BytesIO(csv_file), delimiter='\t', header=None)

urls = df[4]

url_df = spark.createDataFrame(urls.to_frame(name='url'))
url_df.createOrReplaceTempView("temp_table")

current_max_batch = spark.sql("SELECT MAX(batch) AS max_batch FROM infosys.kamikaze_rtsa.bronze_table_raw").first().max_batch
next_batch = current_max_batch + 1 if current_max_batch is not None else 1
df_spark_with_sno = url_df.withColumn("SNo", monotonically_increasing_id()+1)
df_spark_with_sno.withColumn("batch", lit(next_batch)).createOrReplaceTempView("temp")

spark.sql("""
INSERT INTO infosys.kamikaze_rtsa.bronze_table_raw
SELECT 
  SNo, batch, CURRENT_TIMESTAMP() AS Timestamp, url, 0, 0
FROM temp
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
# s3bucket = s3a://workspaces-enb-us-west-2/oregon-prod/7846089482645/FileStore/tables/

In [0]:
%sql
--drop table infosys.kamikaze_rtsa.bronze_table_raw

In [0]:
#%fs rm -r /FileStore/tables/bronze_table

res0: Boolean = true

In [0]:
%sql 
/*
CREATE TABLE infosys.kamikaze_rtsa.bronze_table_raw(
    SNo BIGINT,
    batch BIGINT,
    Timestamp TIMESTAMP,
    url STRING,
    processed SMALLINT, --0 implies article is yet to processed; 1 implies processed
    status  SMALLINT -- 0 is default; 1 is not downloaded ; 2 downloaded
  )USING DELTA
*/

In [0]:
%sql
/*
CREATE EXTERNAL TABLE bronze_table (
    SNo INT,
    batch INT,
    Timestamp TIMESTAMP,
    url STRING,
    processed SMALLINT, --0 implies article is yet to processed; 1 implies processed
    status  SMALLINT -- 0 is default; 1 is not downloaded ; 2 downloaded
  )USING DELTA
location '/FileStore/tables/bronze_table';
*/


com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.SparkSecurityException: [INSUFFICIENT_PERMISSIONS] Insufficient privileges:
User does not have permission CREATE,USAGE on database `default`.
User does not have permission MODIFY on any file.
	at com.databricks.sql.acl.Unauthorized.throwInsufficientPermissionsError(PermissionChecker.scala:336)
	at com.databricks.sql.acl.CheckPermissions$.com$databricks$sql$acl$CheckPermissions$$checkAndAudit(CheckPermissions.scala:2129)
	at com.databricks.sql.acl.CheckPermissions.apply(CheckPermissions.scala:166)
	at com.databricks.sql.acl.CheckPermissions.apply(CheckPermissions.scala:104)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$56(CheckAnalysis.scala:794)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$56$adapted(CheckAnalysis.scala:794)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis0(CheckAnalysis.scala:794)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis0$(CheckAnalysis.scala:169)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis0(Analyzer.scala:301)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:165)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:155)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:155)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:301)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:355)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:379)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:352)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:178)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:348)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$4(QueryExecution.scala:420)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:930)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$2(QueryExecution.scala:420)
	at com.databricks.util.LexicalThreadLocal$Handle.runWith(LexicalThreadLocal.scala:63)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:416)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1038)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:416)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:172)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:172)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:162)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:115)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1038)
	at org.apache.spark.sql.SparkSession.$anonfun$withActiveAndFrameProfiler$1(SparkSession.scala:1045)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.SparkSession.withActiveAndFrameProfiler(SparkSession.scala:1045)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:113)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:841)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1038)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:829)
	at org.apache.spark.sql.SparkSession.sql(SparkSe

In [0]:
#spark.sql("OPTIMIZE bronze_table")

Out[8]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bi

In [0]:
%sql


url https://www.pulse.com.gh/news/politics/cecilia-dapaah-responds-to-claims-of-tainted-assets-amid-ongoing-legal-battle/1nxbbyj https://biztoc.com/x/7c7ba6d2d145c9b4 https://www.clevelandstar.com/news/273934776/mogo-announces-hosting-of-inaugural-mogo-national-championship-event https://biztoc.com/x/2d134e5c35368174 https://www.prnewswire.com/news-releases/isg-takes-root-in-rogers-arkansas-with-a-new-office--acquisition-of-unit-architecture-301903743.html https://biztoc.com/x/418115addc2f94b3 https://biztoc.com/x/e2a4b43538710c5d https://www.wickedlocal.com/story/news/nation/2023/08/17/missing-boy-ikea-brooklyn-new-york/70609182007/ https://www.wsj.com/livecoverage/stock-market-today-dow-jones-08-17-2023/card/cvs-stock-falls-after-insurer-says-it-will-drop-caremark-MUizXB2sBaRLEDzTH9Di https://www.2gb.com/podcast/nights-with-john-stanley-17th-august-3/ https://mix96online.iheart.com/content/2023-08-17-britney-spears-shares-first-post-since-sam-asghari-divorce-news/ https://americansongwriter.com/review-maia-sharp-takes-stock-in-both-kindness-and-compassion-on-reckless-thoughts/ https://www.therakyatpost.com/news/2023/08/17/pahang-state-exco-among-fatalities-of-elmina-crash/ https://techcabal.com/2023/08/17/farm-central-intelligence-emerges-winner-at-afex-code-cash-crop-4-0-ag-hackathon/ https://biztoc.com/x/f85711a24be0ea6c https://newsradio967.iheart.com/content/2023-08-17-watch-cop-takes-down-gun-toting-suspect-by-hitting-them-with-cruiser/ https://www.thecourierexpress.com/ads_to_go/148740-pdf/pdf_54a64d4e-3d0a-11ee-a6a0-9b91acd20b6f.html https://www.manchestereveningnews.co.uk/news/showbiz-news/coronation-street-star-jodie-prenger-27540812 https://www.2gb.com/podcast/tomorrows-headlines-tonight-17th-august-2/ https://money.usnews.com/investing/news/articles/2023-08-17/bae-says-5-5-billion-ball-corp-deal-will-help-it-meet-ukraine-led-defence-demand https://news.yahoo.com/mental-health-investment-compassionate-strategic-150023294.html https://www.shanghainews.net/news/273934832/belgian-zoo-celebrates-birthday-for-giant-panda-twins https://oswegocountytoday.com/lead-story/mayor-barlow-announces-grand-opening-of-10-million-william-s-cahill-pier/ https://theenterpriseleader.com/2023/08/17/gigacloud-technology-nasdaqgct-announces-quarterly-earnings-results.html https://www.bloomberg.com/news/articles/2023-08-17/deerfield-academy-is-raising-89-million-to-build-a-dining-hall https://975now.com/ixp/84/p/buc-ees-michigan-i-75/ https://indianewengland.com/new-ai-model-may-help-screen-for-heart-defect-from-birth/ https://www.eurekalert.org/news-releases/998574 https://www.techcentral.ie/patch-resistant-autonomous-exploits-of-citrix-netscaler-hardware-hit-thousands-in-europe/ https://indianewengland.com/molecules-in-vegetables-like-cauliflower-can-help-to-ease-lung-infection-study/ https://www.anandabazar.com/west-bengal/suvendu-adhikari-and-15-other-bjp-mla-will-go-to-the-house-of-the-dead-student-of-jadavpur-university-dgtl/cid/1452995 https://ticotimes.net/2023/08/17/costa-rica-airfare-deal-denver-fall-and-winter-flights-for-under-400 https://biztoc.com/x/34b7d634da2668bf https://www.alice965.com/2023/08/17/would-you-use-an-a-i-life-coach-google-is-working-on-one/ https://www.theglobeandmail.com/business/article-supreme-court-of-canada-to-hear-appeal-from-airlines-on-air-passenger/ https://www.globenewswire.com/news-release/2023/08/17/2727451/0/en/IZEA-Named-Best-Influencer-Marketing-Company-in-2023-MarTech-Breakthrough-Awards-Program.html https://www.phillytrib.com/news/across_america/texas-woman-charged-with-threatening-to-kill-judge-overseeing-trump-s-federal-election-interference-case/article_3b5e5893-9a29-5ee5-b5e4-608ef8899259.html https://www.atlantanewsfirst.com/2023/08/17/9-months-later-clayton-county-police-continue-search-missing-15-year-old/ https://www.theepochtimes.com/article/the-sinister-theory-behind-the-q-in-lgbtq-5381495 https://biztoc.com/x/9de5a947f789e77b https://www.classiccountry98.com/2023/08/17/fundraiser-

In [0]:
%sql
/*
select processed, status, count(1) 
from infosys.kamikaze_rtsa.bronze_table_raw
group by processed, status
*/

processed status count(1) 0 0 82725 1 2 36717 0 1 2561

In [0]:
%sql
/*
select * from infosys.kamikaze_rtsa.silver_table_processed 
where article_text  == '' and article_title != ''
--where processed==0 and status==1
*/
 

SNo batch Timestamp url article_title article_text transformed_text 19 18 2023-08-18T00:08:31.876+0000 https://m.baidu.com/s?word=%E7%BC%85%E7%94%B8%E5%A6%99%E7%93%A6%E5%BA%95%E6%9C%89%E5%A4%9A%E5%B0%91%E8%AF%88%E9%AA%97%E5%9B%AD%E5%8C%BA&ref=www_colorful&st=111041&tn=iphone&from=1014994m&sa=re_dl_cont_hao123_top&key=509e5369154d4f3ece937b37120c5739 缅甸妙瓦底有多少诈骗园区 3 18 2023-08-18T00:08:31.876+0000 https://www.peoplesworld.org/authors/kevin-ovenden/ Kevin Ovenden 24 19 2023-08-18T00:15:55.034+0000 https://www.marketscreener.com/quote/stock/ADITXT-INC-108790108/news/Aditxt-Inc-NASDAQ-ADTX-Announces-1-for-40-Reverse-Stock-Split-Effective-at-the-Open-of-Trading-44642028/ Redirecting to /quote/stock/ADITXT-INC-108790108/news/CORRECTING-and-REPLACING-Aditxt-Inc-NASDAQ-ADTX-Announces-1-for-40-Reverse-Stock-Split-Effectiv-44642028/ 59 19 2023-08-18T00:15:55.034+0000 https://foxbaltimore.com/morning/former-baltimore-raven-michael-oher-alleges-family-made-millions-off-conservatorship-lie Former Baltimore Raven Michael Oher alleges family made millions off conservatorship lie 6 19 2023-08-18T00:15:55.034+0000 https://www.carsonnow.org/date_list/2023-08-17T09 Events Calendar 16 20 2023-08-18T00:23:22.268+0000 https://wset.com/news/nation-world/firefighter-rescued-from-deadly-baltimore-fire-describes-harrowing-moments-inside-vacant-burning-home-john-mcmaster-baltimore-city-fire-department-two-alarm-fire-shock-trauma-injuries Firefighter rescued from deadly fire describes harrowing moments inside vacant burning hom 1 21 2023-08-18T00:31:11.502+0000 https://www.vavel.com/en-us/nfl/2023/08/17/1153516-cleveland-browns-vs-philadelphia-eagles-live-updates-score-stream-info-lineups-and-how-to-watch-preseason-nfl-2023-match.html Cleveland Browns vs Philadelphia Eagles LIVE: Score Updates (0-0) | 08/17/2023 1 21 2023-08-18T00:31:11.502+0000 https://diariovoces.com.pe/242085/diario-digital-jueves-29 Diario Digital, jueves 17 de agosto 2023 14 22 2023-08-18T00:38:55.389+0000 https://fox11online.com/news/nation-world/florida-man-arrested-for-animal-cruelty-after-deputies-discover-3-malnourished-dogs-zachary-crain-unlawful-confinement-volusia-county-misdemeanor-counts Florida man arrested for animal cruelty after deputies discover 3 malnourished dogs 26 22 2023-08-18T00:38:55.389+0000 https://siliconangle.com/2023/08/17/companies-can-make-use-vector-databases-without-needing-ph-d-ai-cubeconversations/ Companies can make use of vector databases without needing a Ph.D. in AI 47 22 2023-08-18T00:38:55.389+0000 https://siliconangle.com/2023/08/17/arthur-launches-open-source-tool-help-companies-make-data-driven-decisions-llms/ Arthur launches open-source tool to help companies make data-driven decisions about LLMs 26 22 2023-08-18T00:38:55.389+0000 https://abc6onyourside.com/news/nation-world/florida-man-arrested-for-animal-cruelty-after-deputies-discover-3-malnourished-dogs-zachary-crain-unlawful-confinement-volusia-county-misdemeanor-counts Florida man arrested for animal cruelty after deputies discover 3 malnourished dogs 4 23 2023-08-18T00:45:05.922+0000 https://www.snntv.com/clip/15414530/suncoast-weather-authority-forecast Suncoast Weather Authority Forecast 2 24 2023-08-18T00:51:34.121+0000 https://www.archdaily.com/search/projects/min_area/18803/max_area/28205 Projects filtered by area 44 25 2023-08-18T00:57:57.790+0000 https://local21news.com/news/local/manheim-police-searching-for-woman-who-stole-dogs-oxygen-tank Manheim police searching for woman who stole dog's oxygen tank 54 25 2023-08-18T00:57:57.790+0000 https://www.timba.com/blog_posts/san-jos-jazz-friday-part Cuban Music Blogs 35 25 2023-08-18T00:57:57.790+0000 https://carsonnow.org/date_list/2023-08-17T15 Events Calendar 16 26 2023-08-18T01:04:41.763+0000 https://ninernoise.com/posts/niner-noise-podcast-breaks-49ers-first-preseason-game Niner Noise Podcast breaks down 49ers' first preseason game 16 26 2023-08-18T01:04:41.763+0000 https://weartv.com/news/local/escambia-county-grapples-with-pr

In [0]:
%sql
select count(*) from infosys.kamikaze_rtsa.gold_table

count(1) 34779